**Дескриптор** (*descriptor*) – это объект, являющийся атрибутом класса и управляющий доступом к атрибутам экземпляров этого класса. Дескриптор реализует **протокол дескрипторов** (*descriptor protocol*), определяемый методами `__get__()`, `__set__()`, `__delete__()` 
и `__set_name__()`. Дескрипторы лежат в основе работы `property`, `classmethod`, `staticmethod` и **связывания методов** (*method binding*).

Формально дескриптором считается объект, в классе которого реализован хотя бы один из методов `__get__`, `__set__` или `__delete__`. Если определён `__set__` и/или `__delete__`, объект является **дескриптором данных** (*data descriptor*); если определён только `__get__` – **дескриптором не-данных** (*non-data descriptor*). Эти типы различаются приоритетом при поиске атрибутов: data descriptor имеет приоритет над `instance.__dict__`, а non-data descriptor – нет.

Класс, в котором дескриптор определён как атрибут, называется **классом-владельцем** (*owner class*). Дескриптор присваивается атрибуту *класса* напрямую или применяется к методу как декоратор. При создании класса-владельца, у дескриптора вызывается метод `__set_name__()`, которому передается сам класс-владелец и имя атрибута, к которому привязан дескриптор.

## Рабочий пример

Пусть имеется класс для биологических клеток. В этом классе определены два поля объекта:
- `self.cell_type` для хранения типа клетки в виде строки;
- `self.diameter_um` для хранения диаметра клетки.

In [ ]:
class Cell:
    def __init__(self, cell_type, diameter_um):
        self.cell_type = cell_type
        self.diameter_um = diameter_um

Мы хотели бы, чтобы в `self.diameter_um` нельзя было бы установить не числовое значение, и чтобы число было не отрицательным. Это можно сделать через дескриптор.

### Класс дескриптора

Определим класс для дескрипторов `PositiveNumber` для валидации положительных чисел. Этот класс не является непосредственно связанным с классом `Cell`. Мы создаем универсальный дескриптор, который можно создать в любом классе, если нам подходит протокол дескриптора. Протокол определяется определенными в классе `PositiveNumber` методами:

In [ ]:
class PositiveNumber:
    """Дескриптор данных для валидации положительных чисел.
    """
    def __get__(self, instance, owner):
        """Вызывается при чтении атрибута."""
        print(f"вызван метод __get__ дескриптора объекта {instance}, класса {owner}")
        if instance is None:
            return self  # доступ через класс, а не экземпляр
        return getattr(instance, self.storage_name, None)
    
    def __set__(self, instance, value):
        """Вызывается при записи атрибута."""
        print(f"вызван метод __set__ дескриптора объекта {instance}, и передано значение {value}")
        if not isinstance(value, (int, float)):
            raise TypeError(f'{self.name} должен быть числом')
        if value <= 0:
            raise ValueError(f'{self.name} должен быть неотрицательным')
        setattr(instance, self.storage_name, value)
        
    def __set_name__(self, owner, name):
        """Вызывается при создании класса-владельца."""
        print(f"вызван метод __set_name__ дескриптора при создании дескриптора в классе {owner} с именем {name}")
        self.name = name                    # имя атрибута
        self.storage_name = f'__{name}'     # приватное имя для хранения

### Создание дескриптора

Для того, чтобы использовать дескриптор в классе, необходимо создать экземпляр дескриптора и присвоить его атрибуту класса:

In [29]:
class Cell:
    diameter_um = PositiveNumber()  # Дескриптор как атрибут КЛАССА
    def __init__(self, cell_type: str, diameter_um: float):
        self.cell_type = cell_type
        self.diameter_um = diameter_um  # вызовет __set__

вызыван метод __set_name__ дескриптора при создании дескриптора в классе <class '__main__.Cell'> с именем diameter_um


Рассмотрим каждый метод подробнее:

### Метод `__set_name__()`

Метод `__set_name__(self, owner, name)` вызывается автоматически интерпретатором в момент создания класса-владельца, когда дескриптор присваивается атрибуту класса. Метод получает:

- `self` – сам объект-дескриптор;
- `owner` – класс-владелец, в котором дескриптор определён как атрибут;
- `name` – имя атрибута, которому присвоен дескриптор.

Данный дескриптор сохраняет имя поля объекта (`self.name`) для использования в сообщениях об ошибках, а также формирует `self.storage_name` – имя приватного поля экземпляра, в котором будет фактически храниться значение. Это важный паттерн: дескриптор не хранит данные в себе (иначе все экземпляры делили бы одно значение), а использует `__dict__` экземпляра. 

>От того, что в `__set_name__()` определено поле `self.storage_name`, при создании объекта класса-владельца автоматически не возникает поле с именем, которое хранится в `self.storage_name`. Значение `self.storage_name` непосредственно используется методами `__get__()` и `__set__()`.

Когда интерпретатор выполняет определение класса `Cell`, происходит вызов метода дескриптора `diameter_um.__set_name__(Cell, 'diameter_um')`.

### Метод `__set__()`

Метод `__set__()` вызывается при записи значения в атрибут, управляемый дескриптором. 

В нашем случае, метод `__set__()` дескриптора будет вызван при создании экземпляра класса `Cell`, так как в инициализаторе прописаны `__init__()`, в котором прописана инструкция `self.diameter_um = diameter_um`. В этой инструкции идет обращение к имени `self.diameter_um`, которое уже определено в классе и является дескриптором данных. Так как дескриптор данных имеет приоритет выше атрибутов объекта, здесь не будет создано поле объекта `self.diameter_um`: вместо этого интерпретатор обратится к дескриптору. Справа от `self.diameter_um` имеется знак присваивания, поэтому будет вызван метод `__set__(self, instance, value)` дескриптора. 

>`self.cell_type = cell_type` – в этой инструкции просто создается поле объекта `self.cell_type`, которому присваивается значение `cell_type` из аргумента.

Метод получит:

- `self` – объект-дескриптор;
- `instance` – экземпляр класса-владельца, атрибут которого изменяется;
- `value` – присваиваемое значение.

In [6]:
cell = Cell("neuron", 30.0)

вызыван метод __set__ дескриптора объекта <__main__.Cell object at 0x0000025C0976AFC0>, и передано значение 30.0


Что непосредственно делает метод `__set__()` в нашем случае? Вначале производится проверка, является ли значение `value` числом, далее проверяется, является ли число положительным. Если `value` не является числом, генерируется исключение `TypeError`, если `value` является числом, но значение меньше нуля, то генерируется исключение `ValueError`. Если `value` проходит валидацию, то значение присваивается полю объекта `self.__diameter_um`. За это отвечает инструкция `setattr(instance, self.storage_name, value)`. В поле дескриптора `self.storage_name` содержится имя `__diameter_um`. Посмотрим данные объекта `cell`, и убедимся, что в них действительно появлось имя `__diameter_um`:

In [7]:
cell.__dict__

{'cell_type': 'neuron', '__diameter_um': 30.0}

Метод `__set__()` дескриптора будет вызван при любой попытке записи значения в атрибут `diameter_um`:

In [8]:
cell.diameter_um = 15

вызыван метод __set__ дескриптора объекта <__main__.Cell object at 0x0000025C0976AFC0>, и передано значение 15


Если значение не пройдет валидацию, возникнет исключение:

In [9]:
# cell.diameter_um = "15"
# cell.diameter_um = -15

Важно иметь в виду, что создание поля объекта `__diameter_um` выполняется внутри класса `PositiveNumber`, а не внутри `Cell`. Поэтому, `__diameter_um` остается публичным полем, и вообще говоря, может быть изменен напрямую. 

>Дело в том, что механизм искажения имён (*name mangling*) работает только на этапе компиляции класса, когда интерпретатор встречает `__name` в исходном коде класса. Поскольку `setattr(instance, '__diameter_um', value)` выполняется во время выполнения (*runtime*), искажение не применяется, и атрибут остаётся доступным как `instance.__diameter_um`.

In [13]:
cell.__diameter_um = 14

In [14]:
cell.__diameter_um

14

Таким образом имеем фактическое поле объекта `cell.__diameter_um`, с которым пользователь класса `Cell` не взаимодействует явно, и объект дескриптор `Cell.diameter_um`, отвечающий за чтение и запись значений поля `cell.__diameter_um`.

### Метод `__get__()`

Метод дескриптора `__get__()` вызывается при чтении атрибута, управляемого дескриптором. Метод принимает:

- `self` – объект-дескриптор;
- `instance` – экземпляр класса-владельца (или `None`, если доступ производится через класс);
- `owner` – класс-владелец.

In [15]:
cell.diameter_um

вызыван метод __get__ дескриптора объекта <__main__.Cell object at 0x0000025C0976AFC0>, класса <class '__main__.Cell'>


14

В нашей реализации, при вызове метода `__get__()` сначала производится проверка контекста вызова: если `instance is None`, значит, атрибут читается через класс, а не через экземпляр. В этом случае принято возвращать сам дескриптор – это позволяет инспектировать его:

In [26]:
Cell.diameter_um

вызыван метод __get__ дескриптора объекта None, класса <class '__main__.Cell'>


Если атрибут читается через объект, то `instance` будет ссылаться на объект, поэтому выполнится `getattr(instance, self.storage_name, None)` метод `__get__()` вернет результат.

В нашем примере, при  выполнении `cell.diameter_um` интерпретатор вызывает `Cell.diameter_um.__get__(cell, Cell)`.

In [30]:
type(cell).__mro__

(__main__.Cell, object)

## Приоритет поиска атрибутов

При чтении атрибута `obj.attr` применяется **приоритет поиска атрибутов** (*attribute lookup order*):
1. Data descriptor.
1. Instance attribute.
1. Non-data descriptor.
1. Class attribute.

Если нигде не определен атрибут `attr`, вызывается метод `__getattr__()`, если он определен.

При записи атрибута `obj.attr = value` логика приоритета проще:
1. Data descriptor.
1. Instance attribute.

Таким образом, дескриптор данных выше по приоритету, чем атрибут объекта, поэтому, в инструкции `self.diameter_um = diameter_um` инициализатора, интерпретатор обратился к дескриптору данных `diameter_um`, вместо того, чтобы создать атрибут экземпляра, ведь в классе был определен дескриптор данных с именем `diameter_um` как атрибут класса. 

Если бы `diameter_um` был просто атрибутом класса, или же был дескриптором не-данных, то по приоритету поиска интерпретатор создал бы атрибут объекта `self.diameter_um`. Мы не можем присвоить значение атрибуту класса через атрибут объекта.